In [1]:
import json

from paprika.io import json_numpy_obj_hook

## Load Data

In [2]:
with open("potential-decomposition-a000.json", "r") as file:
    potential_bound = json.load(file, object_hook=json_numpy_obj_hook)

with open("potential-decomposition-r014.json", "r") as file:
    potential_unbound = json.load(file, object_hook=json_numpy_obj_hook)

## Estimate Enthalpy

In [3]:
binding_enthalpy = {
    "host-guest": None,
    "complex-solvent": None,
    "solvent-solvent": None,
    "total": None,
}

# Valence terms
valence_bound = (
    potential_bound["bond"].mean()
    + potential_bound["angle"].mean()
    + potential_bound["dihedral"].mean()
)
valence_unbound = (
    potential_unbound["bond"].mean()
    + potential_unbound["angle"].mean()
    + potential_unbound["dihedral"].mean()
)

# total enthalpy
total_bound = valence_bound + potential_bound["nonbonded"].mean()
total_unbound = valence_unbound + potential_unbound["nonbonded"].mean()
binding_enthalpy["total"] = total_bound - total_unbound

# solvent-solvent terms
solvent_bound = potential_bound["solvent"].mean()
solvent_unbound = potential_unbound["solvent"].mean()
binding_enthalpy["solvent-solvent"] = solvent_bound - solvent_unbound

# host-guest solute terms
complex_bound = potential_bound["complex"].mean() + valence_bound
complex_unbound = potential_unbound["complex"].mean() + valence_unbound
binding_enthalpy["host-guest"] = complex_bound - complex_unbound

# solute-solvent terms
binding_enthalpy["complex-solvent"] = binding_enthalpy["total"] - (
    binding_enthalpy["solvent-solvent"] + binding_enthalpy["host-guest"]
)

# Desolvation term
binding_enthalpy["desolvation"] = (
    binding_enthalpy["complex-solvent"] + binding_enthalpy["solvent-solvent"]
)

In [4]:
print(f"Host-Guest Enthalpy  : {binding_enthalpy['host-guest']:>6.2f} kcal/mol")
print(f"Desolvation Enthalpy : {binding_enthalpy['desolvation']:>6.2f} kcal/mol")
print(f"------------------------------------------")
print(f"Total Enthalpy       : {binding_enthalpy['total']:>6.2f} kcal/mol")

Host-Guest Enthalpy  : -40.30 kcal/mol
Desolvation Enthalpy :  14.43 kcal/mol
------------------------------------------
Total Enthalpy       : -25.87 kcal/mol
